In [ ]:
# install.packages("Imap")
rm(list = ls())
library(Imap)
library(dplyr)
library(ggplot2)

setwd("C:/Users/user/Desktop/1091/thesisdata") # desktop

final_Han546 = read.table("final_Han546.txt", header = T)
final_DSM    = read.table(file="yield(grid all).txt",header = T)
fivekmgrid   = read.csv("fivekmgrid.csv")



TY_D = which(final_DSM$lat > 24.589 & final_DSM$lat < 25.116 
           & final_DSM$lon > 120.988 &  final_DSM$lon < 121.480)

TY_H = which(final_Han546$Latitude > 24.589 & final_Han546$Latitude < 25.116 
           & final_Han546$Longitude > 120.988 &  final_Han546$Longitude < 121.480)

TY_DSM = final_DSM[TY_D,]
TY_Han = final_Han546[TY_H,]

L = 3000
# ===== 計算五公里網格點跟DSM有產量的網格點的距離
li = data.frame()
for(i in 1:L){
  for(j in 1:920){
    
    li[i,j] = gdist(fivekmgrid$lat[i],fivekmgrid$lon[i],
                       final_DSM$lat[j],final_DSM$lon[j],units = "km")
  } 
  print(round(i/L,3))
}

# ===== 取出最近的四個點 
x = list()
heads = list()
d = list()
d4 = list()
for(i in 1:L){
  li[i,]  %>% order() -> x[[i]]  # order 從小到大位置
  heads[[i]] = x[[i]][1:4]  #前四的位置
  sort(li[i,])[1:4] -> d4[[i]] # 前四的距離
    
  print(round(i/L,3))
}

da = list()
for(i in 1:L){
   d4[,i] %>% as.numeric() -> da[[i]]
   print(round(i/L,3))
 }

d4 %>% unlist() %>% as.numeric() -> r
matrix(r, nrow = 4) -> rm



# ===== 取出這四個點的產量
yield4 = list()

for(i in 1:L){
  yield4[[i]] = final_DSM$yield[heads[[i]]]
}

fy = c()

# 計算加權產量
for(i in 1:3000){
  pointd1[i] = ( 1/rm[,i][1]) / ( (1/ (rm[,i][1])) + (1/ (rm[,i][2])) + 
                                  (1/ (rm[,i][3])) + (1/ (rm[,i][4]))   )
  pointd2[i] = ( 1/rm[,i][2]) / ( (1/ (rm[,i][1])) + (1/ (rm[,i][2])) + 
                                  (1/ (rm[,i][3])) + (1/ (rm[,i][4]))   )  
  pointd3[i] = ( 1/rm[,i][3]) / ( (1/ (rm[,i][1])) + (1/ (rm[,i][2])) + 
                                  (1/ (rm[,i][3])) + (1/ (rm[,i][4]))   )  
  pointd4[i] = ( 1/rm[,i][4]) / ( (1/ (rm[,i][1])) + (1/ (rm[,i][2])) + 
                                  (1/ (rm[,i][3])) + (1/ (rm[,i][4]))   )  
  
  # 該點的最終產量，依序計算再跟fivekmgrid  cbind 即可，就可以畫圖
  c(pointd1[i],pointd2[i],pointd3[i],pointd4[i] )%*% yield4[[i]] -> fy[i]
  print(round(i/3000,3))
}




cbind(fivekmgrid[1:L,] , fy) -> fiveyield
colnames(fiveyield) = c("lon", "lat", "yield")

col = as.numeric(as.vector(fiveyield$yield))

ggplot(fiveyield, aes(x=lon,y=lat))+
  geom_point( aes(x=lon,y=lat, color = col))+
  scale_colour_gradient(low="red", high = "yellow",name ="Yield")+
  labs(x = "longitude", y = "Latitude")

# ======================= #
#   對應到詳測土壤圖       #
# ======================= #

library(rgdal)
library(raster)
setwd("C:/Users/USER/Desktop/1082/SoilMap/DetailedSoilMap/")  # desktop

datDe = readOGR("DetailedSoilMapWGS84.shp", encoding="UTF-8")

# 取出有在polygon裡面的
inout = over(
  SpatialPoints(fiveyield[,c("lon","lat")],proj4string=CRS(projection(datDe))),
  as(datDe,"SpatialPolygons")
)


inout
cbind(fivekmgrid[1:L,] , inout) -> TWfiveyield
colnames(TWfiveyield) = c("lon", "lat", "yield")



cbind(fivekmgrid[1:L,] , fy, inout)  -> TWfiveyields
TWfiveyields = na.omit(TWfiveyields)
TWfiveyields$inout = TWfiveyields$fy
colnames(TWfiveyields) = c("lon", "lat", "yield", "yield.r")


col = as.numeric(as.vector(TWfiveyields$yield))
ggplot(TWfiveyields, aes(x=lon,y=lat))+
  geom_point( aes(x=lon,y=lat, color = col))+
  scale_colour_gradient(low="red", high = "yellow",name ="Yield")+
  labs(x = "longitude", y = "Latitude")
